# Train and evaluate models

This notebook is meant to standardize model training/evaluation for the base models of the experiments.

It goes through
1. Evaluating saved models on saved dataset
2. Training new models on new dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import numpy as np
import torch

random.seed(0)
np.random.seed(0)
_ = torch.manual_seed(0)
# _ = torch.cuda.manual_seed(42) # only if using GPU

### 1. Evaluating saved models

For now, we create a new test dataset, because the `images.npy` file doesn't contain the labels (circularity, color, area) for the images.

In [3]:
# toggle i to select model to evaluate, toggle n_test to select number of test samples
i = 1
n_test = 1000

Not very high accuracy... Maybe we need to check the tensorflow -> pytorch conversion?

In [4]:
from model_utils import PyTorchCNN, evaluate
from data_utils import create_dataset

# load model
model = PyTorchCNN()
model.load_state_dict(torch.load(f'pytorch_models/cnn_model_{i}.pth'))

# create our own evaluation dataset
images, labels = create_dataset(n_test)
coefficients = np.array([0.4, 0.4, 0.4])

# batch, channel, height, width
X = torch.tensor(images.reshape((-1, 1, 28, 28))).float()
y = torch.tensor(np.matmul(labels, coefficients) > 0.6).float()

# evaluate model
accuracy = evaluate(model, X, y)
print(f'Accuracy: {accuracy:.4f}')

Evaluating: 100%|██████████| 4/4 [00:00<00:00, 42.79it/s]

Accuracy: 0.4990


### 2. Training new models

I'm less familiar with tensorflow, so I created a simple function for training a pytorch model on the ellipse data.

In [5]:
# toggle n_train and n_test to select number of training/evaluation samples
n_train = 50000
n_test = 1000

In [6]:
from model_utils import PyTorchCNN, train

# create our own training dataset
images_train, labels_train = create_dataset(n_train)
coefficients = np.array([0.4, 0.4, 0.4])

# batch, channel, height, width
X_train = torch.tensor(images_train.reshape((-1, 1, 28, 28))).float()
y_train = torch.tensor(np.matmul(labels_train, coefficients) > 0.6).float()

model = PyTorchCNN()
train(model, X_train, y_train, lr=0.0001, num_epochs=5)

Training (Epoch 5): 100%|██████████| 196/196 [00:09<00:00, 20.51it/s, loss=0.193]


The evaluation accuracy looks good! This suggests to me that maybe we're not loading the tensorflow weights correctly? Not sure...

In [7]:
from model_utils import evaluate

# create our own evaluation dataset
images_test, labels_test = create_dataset(n_test)
coefficients = np.array([0.4, 0.4, 0.4])

# batch, channel, height, width
X_test = torch.tensor(images_test.reshape((-1, 1, 28, 28))).float()
y_test = torch.tensor(np.matmul(labels_test, coefficients) > 0.6).float()

accuracy = evaluate(model, X_test, y_test)
print(f'Accuracy: {accuracy:.4f}')

Evaluating: 100%|██████████| 4/4 [00:00<00:00, 45.20it/s]

Accuracy: 0.9100


We'll use this model for the rest of our analysis

In [8]:
torch.save(model.state_dict(), f'pytorch_models/amir_cnn_model.pth')

In [9]:
model = PyTorchCNN()
model.load_state_dict(torch.load(f'pytorch_models/amir_cnn_model.pth'))

evaluate(model, X_test, y_test)

Evaluating: 100%|██████████| 4/4 [00:00<00:00, 48.11it/s]


0.9100000262260437